In [ ]:
class MEME(object):
    def __init__(self, source_data, W, beta = 0.1):
        '''
        source_data will be list of strings
        '''
        
        self.source_data = source_data
        self.W = W
        self.L = 4
        self.beta = beta
        
        # fixed variables
        self.N = None
        self.l = None
        self.n = None
        self.I = None
        self.letter_counts = None
        self.X = list()
        self.beta_i = np.zeros((1,self.L))
        
        # mutable variables, i.e. the value will be changed
        self.p_X1 = None
        self.p_X2 = None
        self.Z = None
        self.z = None
        self.f_ij = None
        self.e_ij = None
        self.lambda1 = 0
        self.lambda2 = 0
        
        self.init_variables()
        
        
    def init_variables(self):
        '''
        initialize the followings: X, f_i, z, I, lambda, N, small L, 
        '''
        
        # record N and small L, and also init beta_i
        self.N = len(self.source_data)
        self.l = list()
        for seq in self.source_data:
            self.l.append(len(seq))
        
        # trans beta_i as frequency and multi with customed beta
        #self.beta_i /= np.sum(self.l)
        #self.beta_i *= self.beta
                
        # init small z
        self.z = np.ones((self.N, max(self.l)))
        for idx in range(self.N):
            self.z[idx][(self.l[idx]- self.W + 1):] = 0
        self.e_ij = self.z.copy()
        
        # init W-mer set X
        # X will be the list of strings
        '''X_ls = list()
        for i in range(self.N):
            X_ls += [ self.source_data[i][j:j+self.W] for j in range( self.l[i] - self.W + 1 ) ]
        self.X = X_ls
        '''
        for i in range(self.N):
            self.X += [ self.source_data[i][j:j+self.W] for j in range(self.l[i] - self.W + 1) ]
        self.n = len(self.X)
        
        # init indicator
        self.I = self.indicator_function()
        
        # init lambda
        lamb_range_min = min( np.sqrt(self.N)/self.n, 1/(2*self.W) )
        lamb_range_max = max( np.sqrt(self.N)/self.n, 1/(2*self.W) )
        self.lambda1 = np.random.uniform(lamb_range_min, lamb_range_max)
        self.lambda2 = 1 - self.lambda1
        
        # init f_ij, size: ( 1 + W ) x L
        self.f_ij = np.zeros( ( (self.W + 1), self.L ) )
        
        # init letter_counts and beta_i
        self.letter_counts = self.count_letter_appearance()
        avg = np.sum(self.letter_counts, axis = 0)/ np.sum(self.letter_counts)
        self.beta_i = self.beta * avg
        self.beta_i = self.beta_i.reshape(1,-1)
        
        # f_0 part
        '''count_num = defaultdict(int)
        for subseq in self.X:
            for s in subseq:
                count_num[s.upper()] += 1
        total = np.concatenate(self.letter_counts).sum()
        for letter in alphabet_dict:
            self.f_ij[0][alphabet_dict[letter] - 1] = count_num[letter]/total
        '''
        self.f_ij[0] = avg
        
        # f_i parts
        for pos in range(self.W):
            C = Counter(self.I.transpose()[pos])
            for val in range(1, self.L + 1):
                self.f_ij[pos + 1][val - 1] = C[val] / self.n
        
        # conditional probabilities p_X1, p_X2
        
        self.p_X1, self.p_X2 = self.condi_distribution(self.f_ij)
        
        return
        
    def indicator_function(self): #done
        '''
        In article, it is the I(k,a) function for eq(7), (8)
        There will transfer the alphabets to the index
        Thus, the results look like 
        [
        [1,2,3,4],
        [4,3,2,1]
        ]

        return
        indicator: transfer input string into tensor indicator, size is n x W
        '''
        assert isinstance(self.X, list), 'Type of X is not list'
        indicator = list()
        for seq in self.X:
            indicator.append(list(map(lambda x: alphabet_dict[x], seq)))

        return np.array(indicator, dtype = 'int')
        
    
    def condi_distribution(self, freq_letter):
        '''
        Calculate the conditional distribution p(Xi | theta_j)
        eq(7),(8) in the MEME article
        To avoid the computation error for the digits, it will use ln() to make it being summation
    
        Arguments:
    
        freq_letter: the frequences for each letter in each position, size: (W + 1) x L
                     background ( 1 x L ) + motif ( W x L )
                     dtype: np.array
        ====================================================
        return: it will be the log form output
    
        p_Xi_1: conditional distribution of motif sequence, size: n x 1
        p_Xi_2: conditional distribution of background, size: n x 1
        '''
    
        p_Xi_1 = np.zeros(self.n)
        p_Xi_2 = np.zeros(self.n)
        freq_letter = np.log(freq_letter)
        f_0 = freq_letter[0]
        f_j = freq_letter[1:]
    
        for subseq in range(self.n):
            for pos in range(self.W):
                p_Xi_1[subseq] += f_j[pos][self.I[subseq][pos] - 1]
                p_Xi_2[subseq] += f_0[self.I[subseq][pos] - 1]

        return np.exp(p_Xi_1), np.exp(p_Xi_2)
    
    def count_letter_appearance(self): #done
        '''
        count the total appearance times for each alphabets
        Arguments:
        data: input source W-mer data, size: n x W
        return:
        count: counting results, size: n x L
        '''

        count = np.zeros( (self.n, 4) , dtype = 'int')

        for i in range(self.n):
            C = Counter(self.X[i])
            count[i][0] = C['A']
            count[i][1] = C['T']
            count[i][2] = C['C']
            count[i][3] = C['G']

        return count
    
    def update_erasing(self):
        '''
        compute the erasing values e_ij
        '''
        
        for i in range(self.N):
            for j in range(self.l[i]):
                lower_bound = j - self.W + 1 if j - self.W + 1 >= 0 else 0
                vals = self.z[i][lower_bound:j]
                if sum(vals) > 1:
                    vals /= sum(vals)
                if any(vals > 9.99999e-1):
                    self.e_ij[i][j] *= 1e-6
                else:
                    self.e_ij[i][j] *= np.exp( sum( np.log( np.ones(len(vals)) - vals ) ) )
                
        return
    
    def update_z(self):
        '''
        update the small z values
        '''
        
        assert sum(self.l)-self.N*(self.W - 1) == self.n, 'Error: index division not equal for updating z.'
        idx = 0
        for i in range(self.N):
            for j in range(self.l[i] - self.W + 1):
                self.z[i][j] = self.Z[idx][0]
                idx += 1
        
        return
    
    def E_step(self): #done
        '''
        calculate the Z_ij, in article's eq(4)
        also update small z depends on the results of Z

        Arguments:
        condi_dis: conditional distribution, from the defined function, size: n x 2
        lamd: probability for using models, size: 1 x 2
        return:
        Z: membership probability, size: n x 2
        '''
        
        # regulate the shape
        p = np.array([self.p_X1, self.p_X2]).transpose()
        lamb = np.array([self.lambda1, self.lambda2]).reshape(1, 2)
        assert p.shape[0] == self.n, 'dimension error for E-step prob'

        multi_results = p * np.tile(lamb, (self.n, 1))
        summation = np.sum(multi_results, axis = 1, keepdims = True)
        Z = multi_results / summation
        self.Z = Z
        #self.update_z()
        
        return Z
    
    def M_step(self):
        '''
        calculate the lambda and f_ij
        
        Arguments:
        Z: membership from E-step, size: n x 2
        I: indicator function, size: n x W
        count: count the appearance time for each alphabet in every sequences, size: n x 4
        
        return:
        
        '''
        
        Z = self.Z.transpose()
        count = self.letter_counts.transpose()

        # update lambda, eq(5)
        lamb = np.mean(Z, axis = 1)
        
        # update f_ij
        # calculate c_0k and c_jk
        c_0k = np.zeros((1, self.L))
        c_jk = np.zeros((self.W, self.L))
        
        # calculate the c_0k
        for k in range(self.L):
            for i in range(self.n):
                for j in range(self.W):
                    c_0k[0][k] += Z[1][i] if self.I[i][j] == (k + 1) else 0
        
        '''for i in range(4):
            c_0k[0][i] = np.sum( Z[1] * count[i] )'''
        
        # calculate the c_jk
        # first make erasing being 1 x n
        E = np.hstack([self.e_ij[i][:(self.l[i] - self.W + 1)] for i in range(self.N)])
        EZ = E * Z[0]
        
        for pos in range(self.W):
            for i in range(self.n):
                c_jk[pos][self.I[i][pos] - 1] += EZ[i]
        
        # start updating f
        # won't directly update self.f since it will use the previous values to check the disparity
        c_0k += self.beta_i
        c_0k /= (np.sum(c_0k, axis = 1) + self.beta)
        
        for pos in range(self.W):
            c_jk[pos] += self.beta_i.squeeze(0)
            c_jk[pos] /= (np.sum(c_jk[pos]) + self.beta)
            
        f_ij = np.vstack((c_0k,c_jk))
        
        #self.update_erasing()
        
        return lamb, f_ij
    
    def update_variables(self, lamb, f_ij):
        '''
        update the variables' values after M step
        it will update: f_ij, lambda, conditional distribution p_X1 and p_X2, erasing
        
        Inputs:
        lamb: lambda, size: 1 x 2
        f_ij: the updated values for position-wise letter frequencies, size: (W+1) x L
        '''
        
        self.lambda1 = lamb[0]
        self.lambda2 = lamb[1]
        self.f_ij = f_ij.copy()
        self.p_X1, self.p_X2 = self.condi_distribution(f_ij)
        #self.update_erasing()
        
        return
    
    def iter(self, epsilon = 1e-6, times = 1000):
        '''
        iterate the E and M steps
        '''
        
        err_ls = list()
        f_ls = list()
        f_ls.append(self.f_ij)
        stt_record = list()
        
        for step in range(times):
            condi_dis = np.array([self.p_X1, self.p_X2])
            lamb = np.array([self.lambda1, self.lambda2])
            stt = time.time()
            new_Z = self.E_step()
            stt_record.append(time.time()-stt)
            stt = time.time()
            new_lambda, new_f_ij = self.M_step()
            stt_record.append(time.time()-stt)
            f_ls.append(new_f_ij)
            
            err = np.sqrt(np.sum( (self.f_ij - new_f_ij) ** 2  , axis = 1))
            err_ls.append(err)
            
            stt = time.time()
            if all(err < epsilon):
                #print('At step {}, EM converges to a solution.'.format(step))
                #print('err is ', err)
                self.update_variables(new_lambda, new_f_ij)
                break
            
            '''if step % 100 == 0 and step > 0:
                print('in step {} err is {}'.format(step, err))
            '''
            self.update_variables(new_lambda, new_f_ij)
            stt_record.append(time.time()-stt)
        
        
        
        return stt_record #err_ls, new_f_ij, f_ls
    
    def show_seq(self):
        '''
        select the motif
        '''
        
        #spec = np.log(self.f_ij[1:]/self.f_ij[0])
        t = np.log((1-self.lambda1)/self.lambda1)
        s = np.log(self.p_X1/self.p_X2)
        motifs = list( compress(self.X, s > t) )
        
        return motifs
    
    def prob_out(self):
        '''
        output the last probability results for motifs
        make the position wise probability based on the passed motifs
        '''
        
        motifs = self.show_seq()
        pos_prob = np.zeros((self.W, self.L))
        logo_tmp = [list() for _ in range(self.W)]
        
        for seq in motifs:
            for i in range(len(seq)):
                pos_prob[i][ alphabet_dict[seq[i]] - 1 ] += 1
                if seq[i] not in logo_tmp[i]:
                    logo_tmp[i].append(seq[i])
        
        pos_prob /= len(motifs)
        
        # sort the logo into order ATCG
        for i in range(len(logo_tmp)):
            if len(logo_tmp[i]) > 1:
                sorted_logo = sorted(list(map(lambda x: alphabet_dict[x], logo_tmp[i])))
                res = list(map(lambda x: val_to_char[x], sorted_logo))
                logo_tmp[i] = res

        logo = list()
        for l in logo_tmp:
            logo.append(''.join(l))
        
        return pos_prob, logo
    